In [5]:
import sys
import os

import pymysql
from IPython.display import clear_output

path_ = os.path.abspath(os.path.join('..'))
sys.path.insert(0, path_)

from env import TARGET_DB
from script import util

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%load_ext sql

clear_output()

20180622 打ち合わせ資料
# データベース構造の改善とデータ投入

## 改善点
- ユニークな識別子(symbolカラム)を付けた。（idカラムはAUTO_INCREMENTで用いることにした）
- 楽天トラベルユーザーレビューのすべてのデータを入れた
- 楽天トラベルユーザーレビュー以外のデータを入れられるよう処理を一般化した
- `travel02_userReview00_20160304.txt`等をバイナリで格納するのをやめた（original_datumテーブルを確保）
- splitter（multi-sentenceをsingle-sentenceにするクラス）の識別子をDBに入れ、別の処理を追加できるようにした

## 今後の予定
- sentenceのinsertがうまくいっていないので確認
- arpaファイルの各n-gramをDBに入れて呼び出す
- DB以外のこともやる

## テーブル一覧

In [6]:
%%sql {util.DATABASE}
show tables;

,Tables_in_sanakin_develop20180618
0,corpora
1,original_datum
2,sentences
3,snkfiles
4,splitters


## 各テーブルの内容

In [11]:
%%sql {util.DATABASE}
select * from corpora limit 10;

,id,name,symbol
0,1,楽天データセット::楽天トラベル::ユーザレビュー,RTUR


In [12]:
%%sql {util.DATABASE}
select * from snkfiles limit 10;

,id,name,corpus_id,checksum
0,1,travel02_userReview00_20160304.txt,1,729a3fae33cb900ed5fb56dffd2e6418790646260cc3b7...
1,2,travel02_userReview01_20160304.txt,1,2c3c9175cfcbb5cfbe421cefcb992cf45c5e1ab8adcda8...
2,3,travel02_userReview02_20160304.txt,1,a40642122df850844d71e4e4b247be5852493a8b7b6369...
3,4,travel02_userReview03_20160304.txt,1,55d5618a17c3843c44920201d51bb65b3aa37daf64c019...
4,5,travel02_userReview04_20160304.txt,1,72126a4027262e00773fc60471aa5013b9f68a3f78490c...
5,6,travel02_userReview05_20160304.txt,1,d7043160735c0d262ac7c69c1b9cf0fe5123317a24e554...
6,7,travel02_userReview06_20160304.txt,1,9984b2882e19e73a8cbf21c6963c20c40362c85a3c2f5b...
7,8,travel02_userReview07_20160304.txt,1,2c434aedb591b22f1f831530edf09b007fb0ead16653a9...
8,9,travel02_userReview08_20160304.txt,1,d9676f6bf3680bfda26151084f62e81e1fc3f9e84752cd...
9,10,travel02_userReview09_20160304.txt,1,6f25bd3bcb2a68e51eb64f13386f4fac41e0e2af3329ad...


In [13]:
%%sql {util.DATABASE}
select * from original_datum limit 10;

,id,snkfile_id,id_in_corpus,contents,symbol
0,1,1,1,このホテルでこの値段は安い。,RTUR00000001
1,2,1,8,駅から出てすぐの所にありました。雨でも安心ですね,RTUR00000008
2,3,1,9,乗車券が有効利用できました,RTUR00000009
3,4,1,12,バスルームが広いのがよかったです。,RTUR00000012
4,5,1,22,ラウンジでコーヒーを頼んだらプチケーキが付いてきた！,RTUR00000022
5,6,1,34,お土産を買うときに便利ですね,RTUR00000034
6,7,1,40,ウォシュレット付！！,RTUR00000040
7,8,1,41,食事がおいしかったです,RTUR00000041
8,9,1,62,JR、地下鉄とも駅に近く、時計台もそば。ただし、すすきのには遠い。歩いて３分ほどで、コンビニ...,RTUR00000062
9,10,1,74,無料バスはＪＲ大阪駅北口(かな?^^;)から１時間に３便出てます(08:00-22:00) ...,RTUR00000074


In [14]:
%%sql {util.DATABASE}
select * from splitters limit 10;

,id,name
0,1,Splitter001


In [15]:
%%sql {util.DATABASE}
select * from sentences limit 10;

""


In [16]:
%%sql {util.DATABASE}
show columns from sentences;

,Field,Type,Null,Key,Default,Extra
0,id,int(11),NO,PRI,None,auto_increment
1,contents,text,NO,,None,
2,data_id,int(11),NO,MUL,None,
3,symbol,varchar(255),NO,UNI,None,
4,splitter_id,int(11),NO,MUL,None,
5,sentence_number,int(11),NO,,None,
6,"sentence_number_max,",int(11),NO,,None,
